In [1]:
import os

In [2]:
!pwd

/home/jovyan/notebooks/executions


In [3]:
import sys
sys.path.append('/home/jovyan/')

In [4]:
from experiments.reporting.report_repository import ReportRepository
from experiments.slack_client import SlackNotifier
from neat.neat_logger import get_neat_logger
from neat.population_engine import EvolutionEngine
from neat.reporting.reports_pyneat import EvolutionReport
from neat.utils import timeit
import math
import random
from itertools import count
import numpy as np

from experiments.logger import logger
from experiments.slack_client import Notifier
from neat.configuration import get_configuration
from neat.evaluation.evaluation_engine import EvaluationStochasticEngine
from neat.evolution_operators.crossover import Crossover
from neat.evolution_operators.mutation import Mutation
from neat.genome import Genome
from neat.reporting.reports_pyneat import EvolutionReport
from neat.species import SpeciationEngine
from neat.stagnation import Stagnation
from neat.population_engine import PopulationEngine
from neat.utils import timeit
import neat
import tests
from config_files.configuration_utils import create_configuration

In [5]:
# dir(neat)

In [6]:
LOGS_PATH = f'{os.getcwd()}/'
logger = get_neat_logger(path=LOGS_PATH)

### Configuration

In [7]:
config_file = 'mnist_downsampled'
config = create_configuration(filename=f'/{config_file}.json')

In [8]:
# TODO: better mechanism for override
config.pop_size = 150
# config.n_samples = 50
#
config.n_generations = 300
config.node_add_prob = 0.7
config.connection_add_prob = 0.7

config.n_output = 3
config.n_processes = 7
config.is_discrete = False
config.compatibility_threshold = 2.5

ALGORITHM_VERSION = 'bayes-neat'
DATASET = config_file
# CORRELATION_ID = 'parameters_grid'
CORRELATION_ID = 'test'

In [9]:
report_repository = ReportRepository.create(project='neuro-evolution', logs_path=LOGS_PATH)
notifier = SlackNotifier.create(channel='batch-jobs')

In [10]:
# evaluation_engine.n_processes

### Initialize

In [11]:
report = EvolutionReport(report_repository=report_repository,
                         algorithm_version=ALGORITHM_VERSION,
                         dataset=DATASET,
                         correlation_id=CORRELATION_ID)

In [12]:
population_engine = PopulationEngine(stagnation_engine=Stagnation())
speciation_engine = SpeciationEngine()
evaluation_engine = EvaluationStochasticEngine()

In [13]:
end_condition = 'normal'
# try:
# initialize population
population = population_engine.initialize_population()
speciation_engine.speciate(population, generation=0)

population = evaluation_engine.evaluate(population=population)

# report
report.report_new_generation(generation=0,
                                  population=population,
                                  species=speciation_engine.species)

2020-03-22 17:50:59,462 - INFO - Generation 0. Best fitness: -85.319. N-Parameters Best: 12
2020-03-22 17:50:59,463 - INFO -                          Mean fitness: -86.705. Mean N-Parameters: 12.0
2020-03-22 17:50:59,463 - INFO -     New best individual (124) found with fitness -85.319
Confusion Matrix:
[[33988 30723 33289]
 [37205 37751 38544]
 [32215 34530 36455]]
Accuracy: 34.380044486812835 %


In [14]:
def run_generation(generation, population):
    
    # create new generation's population
    population = population_engine.reproduce(species=speciation_engine.species,
                                             pop_size=population_engine.pop_size,
                                             generation=generation)
    # create new species based on new population
    speciation_engine.speciate(population, generation=generation)

    # evaluate
    population = evaluation_engine.evaluate(population=population)

    # generation report
    report.report_new_generation(generation=generation,
                                 population=population,
                                 species=speciation_engine.species)
    generation += 1
    return population

In [15]:
def show_species(species):
    species_keys = list(species.keys())
    representative_fitnesses = [round(specie.representative.fitness, 3) for specie in species.values()]
    best_fitnesses = [round(specie.fitness, 3) if specie.fitness else None for specie in species.values()]

    n_genomes = [len(specie.members) for specie in species.values()]
    
    print(f'      keys: {species_keys}')
    print(f'      fitnesses: {representative_fitnesses}')
    print(f'      best fitness: {best_fitnesses}')

    print(f'      n_members: {n_genomes}')

In [16]:
generation = 0

In [17]:
for i in range(50):
    generation += 1
    population = run_generation(generation, population)
    show_species(species=speciation_engine.species)

2020-03-22 17:51:31,760 - INFO - Generation 1. Best fitness: -2.283. N-Parameters Best: 14
2020-03-22 17:51:31,761 - INFO -                          Mean fitness: -44.347. Mean N-Parameters: 14.554
2020-03-22 17:51:31,762 - INFO -     New best individual (232) found with fitness -2.283
Confusion Matrix:
[[18544 48505 30951]
 [21668 56289 35543]
 [18821 48931 35448]]
Accuracy: 35.04321576104226 %
      keys: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
      fitnesses: [-85.759, -86.913, -85.499, -85.282, -86.614, -86.755, -87.069, -87.004, -86.521, -87.706, -86.64, -4.719, -86.607, -86.242]
      best fitness: [-85.319, -85.952, -86.072, -85.99, -86.483, -85.899, -85.321, -85.761, -85.477, -86.939, -87.301, -86.51, -86.906, -86.301]
      n_members: [49, 43, 15, 3, 3, 7, 1, 4, 3, 1, 1, 8, 3, 7]
2020-03-22 17:52:07,849 - INFO - Generation 2. Best fitness: -1.665. N-Parameters Best: 18
2020-03-22 17:52:07,864 - INFO -                          Mean fitness: -16.822. Mean N-Parameters: 

In [18]:
for i in range(150):
    generation += 1
    population = run_generation(generation, population)
    show_species(species=speciation_engine.species)

2020-03-22 18:30:33,388 - INFO - Generation 51. Best fitness: -1.099. N-Parameters Best: 60
2020-03-22 18:30:33,411 - INFO -                          Mean fitness: -2.893. Mean N-Parameters: 47.297
      keys: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
      fitnesses: [-2.022, -3.413, -1.805, -2.022, -1.295, -1.965, -1.113, -2.142, -2.051, -1.817, -1.759, -2.008, -1.702, -1.56, -1.101]
      best fitness: [-1.83, -1.124, -1.103, -1.105, -1.109, -1.105, -1.099, -1.119, -1.531, -1.148, -1.886, -1.225, -1.099, -1.348, -1.101]
      n_members: [7, 12, 20, 16, 12, 13, 5, 9, 3, 12, 2, 2, 1, 8, 23]
2020-03-22 18:31:28,633 - INFO - Generation 52. Best fitness: -1.099. N-Parameters Best: 60
2020-03-22 18:31:28,634 - INFO -                          Mean fitness: -4.448. Mean N-Parameters: 47.613
      keys: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
      fitnesses: [-1.883, -1.955, -1.415, -1.103, -1.719, -1.509, -1.113, -1.136, -1.579, -1.244, -1.764, -1.224, -1.941, -2.049,

In [33]:
show_species(species=speciation_engine.species)

      keys: [1, 2]
      fitnesses: [-1.402, -0.747]
      n_members: [69, 81]


In [19]:
speciation_engine.species[1].representative.fitness

-1.640533208847046

In [20]:
speciation_engine.species[2].representative.fitness

-3.7541580200195312

### Check evaluation results

In [31]:
from neat.evaluation.evaluate_simple import evaluate_genome
from neat.evaluation.utils import get_dataset
from neat.loss.vi_loss import get_loss, get_beta

In [29]:
testing = False

In [33]:
dataset = get_dataset(config.dataset, testing=testing)
loss = get_loss(problem_type=config.problem_type)

GETTING DATASET
